# Project 1 Report

## Arithmetic Intensity Warmup

Arithmetic intensity is the number of operations of a kernel $f(n)$ divided by the number of data items $n$. This follows the formula $\frac{f(n)}{n}$. For this exercise, we measure operations as FLOPs and we assume 8 bytes per floating point number. Additionally, we assume that scalars are loaded into the cache once and then are stored in registers for the remainder of the kernel operations.


| Kernel | Operations | Memory Accesses | Arithmetic Intensity |
| ----------- | ----------- | ----------- | ----------- |
| Y[j] += Y[j] + A[j][i] * B[i] | 2 FLOPs | 32 bytes | 0.0625 FLOPs/byte |
| s += A[i] * A[i] | 2 FLOPs | 8 bytes | 0.25 FLOPs/byte |
| s += A[i] * B[i] | 2 FLOPs | 16 bytes | 0.125 FLOPs/byte |
| Y[i] = A[i] + C*B[i] | 2 FLOPs | 24 bytes | 0.083 FLOPs/byte |

## Part 1: The Roofline Model

| Model | Intel 16 | Intel 18 | AMD 20 |
| ----------- | ----------- | ----------- | ----------- |
| Peak Performance | 30.1 GFLOPs/sec | 43.0 GFLOPs/sec | 40.9 GFLOPs/sec |
| L1 Bandwidth | 171.1 GB/s | 226.2 GB/s | 209.4 GB/s |
| L2 Bandwidth | 112.5 GB/s | 166.7 GB/s | 43.6 GB/s |
| L3 Bandwidth | 72.0 GB/s | 77.8 GB/s | N/A |
| DRAM Bandwidth | 26.5 GB/s | 37.7 GB/s | 37.5 GB/s |
| Ridge Point | ~1.5 FLOPs/byte | ~1.5 FLOPs/byte | ~1.0 FLOPs/byte |

## Part 2: Enter the Agoge